# 03: Structural Similarity - AST vs RK-GST Comparison

This is the **most important** notebook showing our dual structural similarity implementation:
- **AST (Abstract Syntax Tree)**: Deep algorithmic analysis
- **RK-GST (Rabin-Karp Greedy String Tiling)**: Fast copy-paste detection
- **Hybrid Mode**: Combined approach (recommended)

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.similarity.structural import (
    StructuralSimilarity, 
    ASTSimilarityAnalyzer,
    RKGSTSimilarityAnalyzer
)
from src.config.weights import StructuralMethod
from src.io import load_submissions
from src.normalization import get_normalizer

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Understanding the Two Approaches

In [ ]:
print("🔬 Structural Similarity: Two Complementary Approaches")
print("=" * 80)
print("\n1️⃣  AST (Abstract Syntax Tree) - Deep Analysis")
print("   ✓ Analyzes tree structure and control flow")
print("   ✓ Best for: Detecting refactored code, algorithmic similarity")
print("   ✓ Robust to: Variable renaming, code reordering")
print("   ✗ Slower computation time")

print("\n2️⃣  RK-GST (Rabin-Karp Greedy String Tiling) - Pattern Matching")
print("   ✓ Finds maximal matching token sequences")
print("   ✓ Best for: Copy-paste detection with reordering")
print("   ✓ Very fast with rolling hash")
print("   ✗ Less semantic understanding")

print("\n3️⃣  Hybrid Mode - Best of Both Worlds (Recommended)")
print("   ✓ Combines AST (60%) + RK-GST (40%)")
print("   ✓ Comprehensive detection")
print("   ✓ Balanced speed and accuracy")
print("=" * 80)

## 2. Test Case 1: Copy-Paste with Variable Renaming

In [ ]:
# Original code
code_original = '''
def bubble_sort(array):
    n = len(array)
    for i in range(n):
        for j in range(0, n-i-1):
            if array[j] > array[j+1]:
                array[j], array[j+1] = array[j+1], array[j]
    return array
'''

# Plagiarized: just renamed variables
code_renamed = '''
def bubble_sort(arr):
    length = len(arr)
    for x in range(length):
        for y in range(0, length-x-1):
            if arr[y] > arr[y+1]:
                arr[y], arr[y+1] = arr[y+1], arr[y]
    return arr
'''

# Normalize both
normalizer = get_normalizer('python')
norm_original = normalizer.normalize(code_original)
normalizer.reset_counters()
norm_renamed = normalizer.normalize(code_renamed)

# Test both approaches
ast_analyzer = ASTSimilarityAnalyzer()
rkgst_analyzer = RKGSTSimilarityAnalyzer()

ast_score = ast_analyzer.compute_similarity(norm_original, norm_renamed)
rkgst_score = rkgst_analyzer.compute_similarity(norm_original, norm_renamed)

print("🧪 TEST 1: Copy-Paste with Variable Renaming")
print("=" * 70)
print(f"\nAST Score:    {ast_score:.1f}%  {'✅ DETECTED!' if ast_score > 80 else '❌ MISSED'}")
print(f"RK-GST Score: {rkgst_score:.1f}%  {'✅ DETECTED!' if rkgst_score > 80 else '❌ MISSED'}")
print("\n💡 Both should score HIGH - this is blatant plagiarism!")

## 3. Test Case 2: Algorithm Refactoring (Same Logic, Different Structure)

In [ ]:
# Recursive fibonacci
fibonacci_recursive = '''
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)
'''

# Iterative fibonacci - DIFFERENT ALGORITHM
fibonacci_iterative = '''
def fibonacci(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
    return a
'''

# Normalize
normalizer.reset_counters()
norm_recursive = normalizer.normalize(fibonacci_recursive)
normalizer.reset_counters()
norm_iterative = normalizer.normalize(fibonacci_iterative)

# Compare
ast_score2 = ast_analyzer.compute_similarity(norm_recursive, norm_iterative)
rkgst_score2 = rkgst_analyzer.compute_similarity(norm_recursive, norm_iterative)

print("🧪 TEST 2: Different Algorithms (Recursive vs Iterative)")
print("=" * 70)
print(f"\nAST Score:    {ast_score2:.1f}%  {'✅ Correctly LOW' if ast_score2 < 60 else '⚠️ Too high'}")
print(f"RK-GST Score: {rkgst_score2:.1f}%  {'✅ Correctly LOW' if rkgst_score2 < 60 else '⚠️ Too high'}")
print("\n💡 Both should score LOW - these are DIFFERENT valid solutions!")

## 4. Test Case 3: Code Reordering

In [ ]:
# Original order
code_ordered = '''
def helper1(x):
    return x * 2

def helper2(x):
    return x + 10

def main(val):
    temp1 = helper1(val)
    temp2 = helper2(temp1)
    return temp2
'''

# Reordered functions
code_reordered = '''
def main(val):
    temp1 = helper1(val)
    temp2 = helper2(temp1)
    return temp2

def helper2(x):
    return x + 10

def helper1(x):
    return x * 2
'''

# Normalize
normalizer.reset_counters()
norm_ordered = normalizer.normalize(code_ordered)
normalizer.reset_counters()
norm_reordered = normalizer.normalize(code_reordered)

# Compare
ast_score3 = ast_analyzer.compute_similarity(norm_ordered, norm_reordered)
rkgst_score3 = rkgst_analyzer.compute_similarity(norm_ordered, norm_reordered)

print("🧪 TEST 3: Function Reordering")
print("=" * 70)
print(f"\nAST Score:    {ast_score3:.1f}%  {'✅ DETECTED!' if ast_score3 > 70 else '⚠️ Missed'}")
print(f"RK-GST Score: {rkgst_score3:.1f}%  {'✅ DETECTED!' if rkgst_score3 > 70 else '⚠️ Missed'}")
print("\n💡 RK-GST especially good at handling reordering!")

## 5. Comparative Visualization

In [ ]:
# Create comparison chart
test_cases = ['Copy-Paste\n+ Rename', 'Different\nAlgorithms', 'Function\nReordering']
ast_scores = [ast_score, ast_score2, ast_score3]
rkgst_scores = [rkgst_score, rkgst_score2, rkgst_score3]

x = np.arange(len(test_cases))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 7))

bars1 = ax.bar(x - width/2, ast_scores, width, label='AST', 
               color='#3498db', alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax.bar(x + width/2, rkgst_scores, width, label='RK-GST',
               color='#e74c3c', alpha=0.8, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%',
                ha='center', va='bottom', fontsize=11, fontweight='bold')

# Add threshold lines
ax.axhline(y=90, color='red', linestyle='--', alpha=0.5, linewidth=2, label='Severe (90%)')
ax.axhline(y=60, color='orange', linestyle='--', alpha=0.5, linewidth=2, label='Partial (60%)')

ax.set_ylabel('Similarity Score (%)', fontsize=13, fontweight='bold')
ax.set_xlabel('Test Case', fontsize=13, fontweight='bold')
ax.set_title('AST vs RK-GST: Side-by-Side Comparison', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(test_cases, fontsize=11)
ax.legend(fontsize=11, loc='upper right')
ax.set_ylim(0, 110)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Hybrid Mode - Best of Both Worlds

In [ ]:
# Test hybrid mode
hybrid_analyzer = StructuralSimilarity(method=StructuralMethod.HYBRID)

print("🎯 HYBRID MODE ANALYSIS")
print("=" * 70)

# Test all three cases
test_pairs = [
    ('Copy-Paste + Rename', norm_original, norm_renamed),
    ('Different Algorithms', norm_recursive, norm_iterative),
    ('Function Reordering', norm_ordered, norm_reordered)
]

for name, code1, code2 in test_pairs:
    result = hybrid_analyzer.compute_similarity(code1, code2)
    print(f"\n{name}:")
    print(f"  Final Score: {result['score']:.1f}%")
    if result['breakdown']:
        print(f"  Breakdown: AST={result['breakdown']['ast']:.1f}%, "
              f"RK-GST={result['breakdown']['rkgst']:.1f}%")
    
    if result['score'] >= 90:
        verdict = "🚨 SEVERE PLAGIARISM"
    elif result['score'] >= 60:
        verdict = "⚠️  PARTIAL SIMILARITY"
    else:
        verdict = "✅ CLEAN"
    print(f"  Verdict: {verdict}")

## 7. Real Dataset Analysis

In [ ]:
# Load sample submissions
submissions = load_submissions('../data/raw/sample_submissions.csv')

# Normalize all
normalized_codes = []
for sub in submissions:
    normalizer.reset_counters()
    normalized_codes.append(normalizer.normalize(sub['code']))

# Compute similarity matrices for both methods
n = len(submissions)
ast_matrix = np.zeros((n, n))
rkgst_matrix = np.zeros((n, n))
hybrid_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if i == j:
            ast_matrix[i][j] = 100
            rkgst_matrix[i][j] = 100
            hybrid_matrix[i][j] = 100
        elif i < j:
            ast_matrix[i][j] = ast_analyzer.compute_similarity(
                normalized_codes[i], normalized_codes[j])
            rkgst_matrix[i][j] = rkgst_analyzer.compute_similarity(
                normalized_codes[i], normalized_codes[j])
            result = hybrid_analyzer.compute_similarity(
                normalized_codes[i], normalized_codes[j])
            hybrid_matrix[i][j] = result['score']
            
            # Mirror
            ast_matrix[j][i] = ast_matrix[i][j]
            rkgst_matrix[j][i] = rkgst_matrix[i][j]
            hybrid_matrix[j][i] = hybrid_matrix[i][j]

print(f"✓ Computed {n}x{n} similarity matrices for all three methods")

## 8. Three-Way Heatmap Comparison

In [ ]:
submission_ids = [sub['submission_id'] for sub in submissions]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# AST heatmap
sns.heatmap(ast_matrix, annot=True, fmt='.0f', cmap='RdYlGn',
            vmin=0, vmax=100, xticklabels=submission_ids,
            yticklabels=submission_ids, ax=axes[0], cbar_kws={'label': 'Score (%)'})
axes[0].set_title('AST Method', fontsize=14, fontweight='bold', pad=10)

# RK-GST heatmap
sns.heatmap(rkgst_matrix, annot=True, fmt='.0f', cmap='RdYlGn',
            vmin=0, vmax=100, xticklabels=submission_ids,
            yticklabels=submission_ids, ax=axes[1], cbar_kws={'label': 'Score (%)'})
axes[1].set_title('RK-GST Method', fontsize=14, fontweight='bold', pad=10)

# Hybrid heatmap
sns.heatmap(hybrid_matrix, annot=True, fmt='.0f', cmap='RdYlGn',
            vmin=0, vmax=100, xticklabels=submission_ids,
            yticklabels=submission_ids, ax=axes[2], cbar_kws={'label': 'Score (%)'})
axes[2].set_title('HYBRID Method (60% AST + 40% RK-GST)', 
                  fontsize=14, fontweight='bold', pad=10)

plt.suptitle('Structural Similarity: Three-Method Comparison', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 9. Method Agreement Analysis

In [ ]:
# Compare where methods agree/disagree
differences = []
for i in range(n):
    for j in range(i+1, n):
        diff = abs(ast_matrix[i][j] - rkgst_matrix[i][j])
        differences.append({
            'pair': f"{submission_ids[i]}-{submission_ids[j]}",
            'ast': ast_matrix[i][j],
            'rkgst': rkgst_matrix[i][j],
            'diff': diff
        })

# Sort by difference
differences.sort(key=lambda x: x['diff'], reverse=True)

print("📊 Method Agreement Analysis")
print("=" * 70)
print(f"\n{'Pair':<15} {'AST':<10} {'RK-GST':<10} {'Difference':<12}")
print("=" * 70)

for item in differences:
    agreement = "✅ Agree" if item['diff'] < 10 else "⚠️  Disagree"
    print(f"{item['pair']:<15} {item['ast']:<10.1f} {item['rkgst']:<10.1f} "
          f"{item['diff']:<12.1f} {agreement}")

avg_diff = np.mean([d['diff'] for d in differences])
print(f"\nAverage difference: {avg_diff:.1f}%")
print(f"\n💡 Smaller differences = Both methods agree → Higher confidence!")

## Summary

### AST (Abstract Syntax Tree)
✅ **Strengths**: Deep algorithmic understanding, robust to refactoring
❌ **Weaknesses**: Slower, more complex
🎯 **Best for**: Detecting sophisticated plagiarism attempts

### RK-GST (Rabin-Karp Greedy String Tiling)
✅ **Strengths**: Fast, handles reordering well
❌ **Weaknesses**: Less semantic understanding
🎯 **Best for**: Quick screening, copy-paste detection

### Hybrid Mode (RECOMMENDED)
✅ **Strengths**: Combines both approaches, balanced accuracy
✅ **Comprehensive**: Catches both copy-paste AND refactored plagiarism
🎯 **Best for**: Production use in academic settings

**Key Insight**: Using BOTH methods gives us confidence when they agree and alerts us to investigate when they disagree significantly!

**Next Steps**: Proceed to notebook 04 for semantic similarity analysis!